In [1]:
from datasets import load_dataset

In [38]:
dataset = load_dataset("dwisaji/indonesia-telecomunication-sentiment-dataset")

Using custom data configuration dwisaji--indonesia-telecomunication-sentiment-dataset-835198df64d84c97
Reusing dataset csv (C:\Users\W I N D O W S\.cache\huggingface\datasets\dwisaji___csv\dwisaji--indonesia-telecomunication-sentiment-dataset-835198df64d84c97\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [39]:
dataset["train"][100]

{'text': 'ko lemot bgt si', 'label': 2}

In [40]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("cahya/bert-base-indonesian-522M")

loading configuration file https://huggingface.co/cahya/bert-base-indonesian-522M/resolve/main/config.json from cache at C:\Users\W I N D O W S/.cache\huggingface\transformers\3ef083ba4a7b9a8046971990410002f446e0cfdbf2daf9b58f773550b8313214.a3ba869a2dcc464ebdde3e20c0b89e8b26778b52208795084484baa6b7846820
Model config BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-522M",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading file https:

In [51]:
max_length = 512
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,max_length=max_length)

In [52]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(200))

  0%|          | 0/3 [00:00<?, ?ba/s]

Loading cached shuffled indices for dataset at C:\Users\W I N D O W S\.cache\huggingface\datasets\dwisaji___csv\dwisaji--indonesia-telecomunication-sentiment-dataset-835198df64d84c97\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a\cache-b82d7162b7cabb93.arrow


In [53]:
from transformers import  AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("cahya/bert-base-indonesian-522M",num_labels=3)

loading configuration file https://huggingface.co/cahya/bert-base-indonesian-522M/resolve/main/config.json from cache at C:\Users\W I N D O W S/.cache\huggingface\transformers\3ef083ba4a7b9a8046971990410002f446e0cfdbf2daf9b58f773550b8313214.a3ba869a2dcc464ebdde3e20c0b89e8b26778b52208795084484baa6b7846820
Model config BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-522M",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_

In [54]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="ModelCP",num_train_epochs=5,evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [55]:
import numpy as np
from datasets import load_metric
from  datasets import list_metrics
metric = load_metric("accuracy")

In [56]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [57]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [58]:
trainer.train()
print("done")


The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1000
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 625


Epoch,Training Loss,Validation Loss



KeyboardInterrupt

